# Getting Started with Miyagi's Retrieval Augmented Generation (RaG) Workflow using Azure Cognitive Search (ACS) and Semantic Kernel

To quickly get started, follow these steps:

1. Ensure the [Polyglot notebooks extension](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) is installed.
2. [Create a new Azure OpenAI service (or use an existing OpenAI service)](https://learn.microsoft.com/en-us/azure/ai-services/openai/chatgpt-quickstart?tabs=command-line&pivots=programming-language-studio#prerequisites).
3. [Deploy the `gpt-35-turbo` and `text-embeddings-ada-002` models](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models#working-with-models).
4. [Create an Azure Cognitive Search instance and enable the Semantic Search capability](https://learn.microsoft.com/en-us/azure/search/semantic-search-overview#enable-semantic-search).
5. Copy the `.env.example` file from the `rag` folder to `dotnet/.env` and paste the corresponding values from the resources you provisioned in the earlier steps.
6. Click on `Run All`.

> This guide is based on [Devis' ACS notebooks](https://github.com/dluc/Azure-Cognitive-Search-20230720)

> You will need an [.Net 7 SDK](https://dotnet.microsoft.com/en-us/download) and [Polyglot](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) to get started with this notebook using .Net Interactive

## Load settings from .env file

In [7]:
#r "nuget: dotenv.net, 3.1.2"
dotenv.net.DotEnv.Load();
var env = dotenv.net.DotEnv.Read();

Installed Packages dotenv.net, 3.1.2

## Prepare kernel using Azure Cognitive Search

In [8]:
#r "nuget: Microsoft.SemanticKernel, 0.17.230718.1-preview"
#r "nuget: Microsoft.SemanticKernel.Connectors.Memory.AzureSearch, 0.17.230718.1-preview"

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.Memory.AzureSearch;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Text;
using System.IO;
using System.Text.Json;

var kernel = Kernel.Builder
    
    // Use Azure Cognitive Search for the kernel Memory
    .WithMemoryStorage(new AzureSearchMemoryStore(
        env["AZURE_SEARCH_ENDPOINT"],
        env["AZURE_SEARCH_API_KEY"]))

    // Use Azure OpenAI for Embeddings (model: text-embedding-ada-002)
    .WithAzureTextEmbeddingGenerationService(
        deploymentName: "text-embedding-ada-002",
        endpoint: env["AZURE_OPENAI_ENDPOINT"],
        apiKey: env["AZURE_OPENAI_API_KEY"])

    // Use Azure OpenAI for Semantic Functions (model = gpt-35-turbo)
    .WithAzureChatCompletionService(
        deploymentName: "gpt-35-turbo",
        endpoint: env["AZURE_OPENAI_ENDPOINT"],
        apiKey: env["AZURE_OPENAI_API_KEY"])
        
    .Build();

Installed Packages Microsoft.SemanticKernel, 0.17.230718.1-preview Microsoft.SemanticKernel.Connectors.Memory.AzureSearch, 0.17.230718.1-preview

## Vectorize and persist embeddings in Azure Cognitive Search with Semantic Kernel

In [9]:
var dataset = "intelligent-investor.txt";
var recommendationServicePath = "../../../../services/recommendation-service/dotnet";
const int MaxTokensPerParagraph = 160;
const int MaxTokensPerLine = 60;

// Read file from local file system
var filePath = Path.Combine(recommendationServicePath, "resources", "sample-datasets", dataset);
var streamReader = new StreamReader(filePath);
var text = await streamReader.ReadToEndAsync();

// Chunk, generate embeddings, and persist to vectordb
var memoryCollectionName = "userId";

var lines = TextChunker.SplitPlainTextLines(text, MaxTokensPerLine);
var chunks = TextChunker.SplitPlainTextParagraphs(lines, MaxTokensPerParagraph);

for (var i = 0; i < chunks.Count; i++)
{
    var chunk = chunks[i];
    var key = await kernel.Memory.SaveInformationAsync(
        memoryCollectionName,
        chunk,
        $"{dataset}-{i}",
        $"Dataset: {dataset} Chunk: {i}",
        i.ToString());
}
System.Console.WriteLine($"Saved {chunks.Count} chunks to memory collection {memoryCollectionName}");

Saved 46 chunks to memory collection userId


## Search and retrieve documents using Semantic Kernel

In [10]:
var query = "Ben Graham's investment philosophy";
Console.WriteLine(query + "\n");

var results = kernel.Memory.SearchAsync(collection: memoryCollectionName, query, limit: 2);
await foreach(var result in results)
{
    Console.WriteLine("   " + result.Metadata.Text);
    Console.WriteLine("   Relevance: " + result.Relevance + "\n");
}

Ben Graham's investment philosophy

   Graham designed his approach to craft a reliable portfolio that requires minimum maintenance and offers maximum odds of a steady return.
By sticking to a formula for investment decisions, the defensive investor gives up the risk of speculating on stock movements and gains steady returns.
After the initial curation, if the investor trades more than twice a year, it is a clear sign something has gone wrong.
[widget]
SUMMARY
Price and value are two different concepts, and stock prices frequently don't reflect a company's actual value. More risk does not necessarily correlate with more gain.
   Relevance: 0.88120747

   Projection is dangerous because the future is uncertain, and inflation, economic recessions, pandemics and geopolitical upheavals often arrive without warning.
Graham argues that it is a fool's errand for an ordinary investor to attempt making money by timing the market.
Graham advocates for a protection-based approach that does not tr

## Grounding Miyagi prompts with SK's Memory "recall"

> Note that this prompt template (semantic function) is located under services/reccommendation-service/dotnet/plugins/AdvisorPlugin

In [11]:
using Microsoft.SemanticKernel.Skills.Core;
using Microsoft.SemanticKernel.SkillDefinition;

// recall is from the TextMemorySkill, which does the retrieval step
kernel.ImportSkill(new TextMemorySkill());

var pluginFolder = $"{recommendationServicePath}/plugins";
var advisorPlugin = kernel.ImportSemanticSkillFromDirectory(pluginFolder, "AdvisorPlugin");
advisorPlugin

key value InvestmentAdvise {"Name":"InvestmentAdvise","SkillName":"AdvisorPlugin","Description":"Gives financial advise on how to allocate portfolio, given a risk tolerance and a set of stocks","IsSemantic":true,"RequestSettings":{"Temperature":0.9,"TopP":1,"PresencePenalty":1,"FrequencyPenalty":1,"MaxTokens":512,"StopSequenc... Name InvestmentAdvise SkillName AdvisorPlugin Description Gives financial advise on how to allocate portfolio, given a risk tolerance and a set of stocks IsSemantic True RequestSettings Microsoft.SemanticKernel.AI.TextCompletion.CompleteRequestSettings Temperature 0.9 TopP 1 PresencePenalty 1 FrequencyPenalty 1 MaxTokens 512 StopSequences ResultsPerPrompt 1 ChatSystemPrompt Assistant is a large language model. TokenSelectionBiases (empty) Parameters index value 0 Microsoft.SemanticKernel.SkillDefinition.ParameterView Name portfolio Description Asset portfolio as JSON text DefaultValue 1 Microsoft.SemanticKernel.SkillDefinition.ParameterView Name user Description User information as JSON text DefaultValue 2 Microsoft.SemanticKernel.SkillDefinition.ParameterView Name voice Description DefaultValue 3 Microsoft.SemanticKernel.SkillDefinition.ParameterView Name risk Description DefaultValue 4 Microsoft.SemanticKernel.SkillDefinition.ParameterView Name bingResults Description DefaultValue 5 Microsoft.SemanticKernel.SkillDefinition.ParameterView Name stocks Description DefaultValue PortfolioAllocation {"Name":"PortfolioAllocation","SkillName":"AdvisorPlugin","Description":"Gives financial advise on how to allocate assets, given a risk tolerance","IsSemantic":true,"RequestSettings":{"Temperature":0.8,"TopP":1,"PresencePenalty":0,"FrequencyPenalty":0,"MaxTokens":512,"StopSequences":[],"ResultsPerPr... Name PortfolioAllocation SkillName AdvisorPlugin Description Gives financial advise on how to allocate assets, given a risk tolerance IsSemantic True RequestSettings Microsoft.SemanticKernel.AI.TextCompletion.CompleteRequestSettings Temperature 0.8 TopP 1 PresencePenalty 0 FrequencyPenalty 0 MaxTokens 512 StopSequences ResultsPerPrompt 1 ChatSystemPrompt Assistant is a large language model. TokenSelectionBiases (empty) Parameters index value 0 Microsoft.SemanticKernel.SkillDefinition.ParameterView Name portfolio Description Asset portfolio as JSON text DefaultValue 1 Microsoft.SemanticKernel.SkillDefinition.ParameterView Name user Description User information as JSON text DefaultValue 2 Microsoft.SemanticKernel.SkillDefinition.ParameterView Name risk Description DefaultValue

### Set context variables

In [20]:
var context = kernel.CreateNewContext();

// Set the parameters for the TextMemorySkill
context[TextMemorySkill.CollectionParam] = memoryCollectionName;
context[TextMemorySkill.RelevanceParam] = "0.8";
context[TextMemorySkill.LimitParam] = "3";

// Set the parameters for the AdvisorPlugin
var stocks = new[] {
    new {symbol = "MSFT", allocation = 0.3},
    new {symbol = "ACN", allocation = 0.1},
    new {symbol = "JPM", allocation = 0.3},
    new {symbol = "PEP", allocation = 0.3}
};
context["stocks"] = JsonSerializer.Serialize(stocks);

context["userId"] = "50";
context["voice"] = "Jim Cramer";
context["risk"] = "aggressive";

context

Result ErrorOccurred False LastErrorDescription LastException <null> ModelResults CancellationToken System.Threading.CancellationToken IsCancellationRequested False CanBeCanceled False WaitHandle System.Threading.ManualResetEvent Handle 3856 SafeWaitHandle Microsoft.Win32.SafeHandles.SafeWaitHandle IsInvalid False IsClosed False Culture en-US Parent en Parent Parent Parent Parent LCID 127 KeyboardLayoutId 127 Name IetfLanguageTag DisplayName Invariant Language (Invariant Country) NativeName Invariant Language (Invariant Country) EnglishName Invariant Language (Invariant Country) TwoLetterISOLanguageName iv ThreeLetterISOLanguageName ivl ThreeLetterWindowsLanguageName IVL CompareInfo CompareInfo - TextInfo TextInfo - IsNeutralCulture False CultureTypes SpecificCultures, InstalledWin32Cultures NumberFormat System.Globalization.NumberFormatInfo DateTimeFormat System.Globalization.DateTimeFormatInfo Calendar System.Globalization.GregorianCalendar OptionalCalendars [ System.Globalization.GregorianCalendar ] UseUserOverride False IsReadOnly True LCID 127 KeyboardLayoutId 127 Name IetfLanguageTag DisplayName Invariant Language (Invariant Country) NativeName Invariant Language (Invariant Country) EnglishName Invariant Language (Invariant Country) TwoLetterISOLanguageName iv ThreeLetterISOLanguageName ivl ThreeLetterWindowsLanguageName IVL CompareInfo CompareInfo - Name Version System.Globalization.SortVersion LCID 127 TextInfo TextInfo - ANSICodePage 1252 OEMCodePage 437 MacCodePage 10000 EBCDICCodePage 37 LCID 127 CultureName IsReadOnly True ListSeparator , IsRightToLeft False IsNeutralCulture False CultureTypes SpecificCultures, InstalledWin32Cultures NumberFormat System.Globalization.NumberFormatInfo CurrencyDecimalDigits 2 CurrencyDecimalSeparator . IsReadOnly True CurrencyGroupSizes [ 3 ] NumberGroupSizes [ 3 ] PercentGroupSizes [ 3 ] CurrencyGroupSeparator , CurrencySymbol ¤ NaNSymbol NaN CurrencyNegativePattern 0 NumberNegativePattern 1 PercentPositivePattern 0 PercentNegativePattern 0 NegativeInfinitySymbol -Infinity NegativeSign - NumberDecimalDigits 2 NumberDecimalSeparator . NumberGroupSeparator , CurrencyPositivePattern 0 PositiveInfinitySymbol Infinity PositiveSign + PercentDecimalDigits 2 PercentDecimalSeparator . PercentGroupSeparator , PercentSymbol % PerMilleSymbol ‰ NativeDigits [ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9 ] DigitSubstitution None DateTimeFormat System.Globalization.DateTimeFormatInfo AMDesignator AM Calendar System.Globalization.GregorianCalendar DateSeparator / FirstDayOfWeek Sunday CalendarWeekRule FirstDay FullDateTimePattern dddd, dd MMMM yyyy HH:mm:ss LongDatePattern dddd, dd MMMM yyyy LongTimePattern HH:mm:ss MonthDayPattern MMMM dd PMDesignator PM RFC1123Pattern ddd, dd MMM yyyy HH':'mm':'ss 'GMT' ShortDatePattern MM/dd/yyyy ShortTimePattern HH:mm SortableDateTimePattern yyyy'-'MM'-'dd'T'HH':'mm':'ss TimeSeparator : UniversalSortableDateTimePattern yyyy'-'MM'-'dd HH':'mm':'ss'Z' YearMonthPattern yyyy MMMM AbbreviatedDayNames [ Sun, Mon, Tue, Wed, Thu, Fri, Sat ] ShortestDayNames [ Su, Mo, Tu, We, Th, Fr, Sa ] DayNames [ Sunday, Monday, Tuesday, Wednesday, Thursday, Friday, Saturday ] AbbreviatedMonthNames [ Jan, Feb, Mar, Apr, May, Jun, Jul, Aug, Sep, Oct, Nov, Dec, ] MonthNames [ January, February, March, April, May, June, July, August, September, October, November, December, ] IsReadOnly True NativeCalendarName Gregorian Calendar AbbreviatedMonthGenitiveNames [ Jan, Feb, Mar, Apr, May, Jun, Jul, Aug, Sep, Oct, Nov, Dec, ] MonthGenitiveNames [ January, February, March, April, May, June, July, August, September, October, November, December, ] Calendar System.Globalization.GregorianCalendar MinSupportedDateTime 1/1/0001 12:00:00 AM MaxSupportedDateTime 12/31/9999 11:59:59 PM AlgorithmType SolarCalendar CalendarType Localized Eras [ 1 ] TwoDigitYearMax 2029 IsReadOnly True OptionalCalendars index value 0 System.Globalization.GregorianCalendar UseUserOverride False IsReadOnly True LCID 127 KeyboardLayoutId 12

### Create native function

In [22]:
using System.ComponentModel;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.SkillDefinition;

/// <summary>
///     UserProfilePlugin shows a native skill example to look user info given userId.
/// </summary>
/// <example>
///     Usage: kernel.ImportSkill("UserProfilePlugin", new UserProfilePlugin());
///     Examples:
///     SKContext["userId"] = "000"
///     {{UserProfilePlugin.GetUserAge $userId }} => {userProfile}
/// </example>
public class UserProfilePlugin
{
    /// <summary>
    ///     Name of the context variable used for UserId.
    /// </summary>
    public const string UserId = "UserId";

    private const string DefaultUserId = "40";
    private const int DefaultAnnualHouseholdIncome = 150000;
    private const int Normalize = 81;

    /// <summary>
    ///     Lookup User's age for a given UserId.
    /// </summary>
    /// <example>
    ///     SKContext[UserProfilePlugin.UserId] = "000"
    /// </example>
    /// <param name="context">Contains the context variables.</param>
    [SKFunction]
    [SKName("GetUserAge")]
    [Description("Given a userId, get user age")]
    public string GetUserAge(
        [Description("Unique identifier of a user")]
        string userId,
        SKContext context)
    {
        // userId = context.Variables.ContainsKey(UserId) ? context[UserId] : DefaultUserId;
        userId = string.IsNullOrEmpty(userId) ? DefaultUserId : userId;

        int age;

        if (int.TryParse(userId, out var parsedUserId))
            age = parsedUserId > 100 ? parsedUserId % Normalize : parsedUserId;
        else
            age = int.Parse(DefaultUserId);

        // invoke a service to get the age of the user, given the userId
        return age.ToString();
    }

    /// <summary>
    ///     Lookup User's annual income given UserId.
    /// </summary>
    /// <example>
    ///     SKContext[UserProfilePlugin.UserId] = "000"
    /// </example>
    /// <param name="context">Contains the context variables.</param>
    [SKFunction]
    [SKName("GetAnnualHouseholdIncome")]
    [Description("Given a userId, get user annual household income")]
    public string GetAnnualHouseholdIncome(
        [Description("Unique identifier of a user")]
        string userId,
        SKContext context)
    {
        // userId = context.Variables.ContainsKey(UserId) ? context[UserId] : DefaultUserId;
        userId = string.IsNullOrEmpty(userId) ? DefaultUserId : userId;

        var random = new Random();
        var randomMultiplier = random.Next(1000, 8000);

        // invoke a service to get the annual household income of the user, given the userId
        var annualHouseholdIncome = int.TryParse(userId, out var parsedUserId)
            ? parsedUserId * randomMultiplier
            : DefaultAnnualHouseholdIncome;

        return annualHouseholdIncome.ToString();
    }
}

In [23]:
// import the UserProfilePlugin
kernel.ImportSkill(new UserProfilePlugin(), "UserProfilePlugin");

### Invoke the LLM

In [24]:
var result = await kernel.Func("AdvisorPlugin", "InvestmentAdvise").InvokeAsync("Are growth stocks better, according to Ben Graham?", context);

Console.WriteLine(result);


{
  "portfolio": [
    {"symbol":"MSFT","gptRecommendation":"Buy and hold. Tech giant with great potential."},
    {"symbol":"ACN","gptRecommendation":"Consulting powerhouse with steady growth, keep it in your portfolio."},
    {"symbol":"JPM","gptRecommendation":"Top-performing bank stock, stay invested for long-term gains."},
    {"symbol":"PEP","gptRecommendation":"Consumer staple that offers stable dividends, continue to hold."}
  ]
}


![RaG Workflow](../../../../assets/images/sk-memory-orchestration.png)